# Inference on Predictive and Causal Effects in High-Dimensional Nonlinear Models

## Impact of 401(k) on Financial Wealth

As a practical illustration of the methods developed in this lecture, we consider estimation of the effect of 401(k) eligibility and participation on accumulated assets. 401(k) plans are pension accounts sponsored by employers. The key problem in determining the effect of participation in 401(k) plans on accumulated assets is saver heterogeneity coupled with the fact that the decision to enroll in a 401(k) is non-random. It is generally recognized that some people have a higher preference for saving than others. It also seems likely that those individuals with high unobserved preference for saving would be most likely to choose to participate in tax-advantaged retirement savings plans and would tend to have otherwise high amounts of accumulated assets. The presence of unobserved savings preferences with these properties then implies that conventional estimates that do not account for saver heterogeneity and endogeneity of participation will be biased upward, tending to overstate the savings effects of 401(k) participation.

One can argue that eligibility for enrolling in a 401(k) plan in this data can be taken as exogenous after conditioning on a few observables of which the most important for their argument is income. The basic idea is that, at least around the time 401(k)’s initially became available, people were unlikely to be basing their employment decisions on whether an employer offered a 401(k) but would instead focus on income and other aspects of the job.

### Data

The data set comes from the `hdm` R package, and is loaded in Julia using the `RData` package

In [1]:
using RData, CSV, DataFrames, StatsBase, Gadfly, StatsModels, GLM

In [2]:
rdata = RData.load("../data/pension.RData", convert = true);

In [3]:
data = DataFrame(rdata["pension"]);
categorical!(data, [:e401, :p401]);

The data consist of 9,915 observations at the household level drawn from the 1991 Survey of Income and Program Participation (SIPP).  All the variables are referred to 1990. We use net financial assets (*net\_tfa*) as the outcome variable, $Y$,  in our analysis. The net financial assets are computed as the sum of IRA balances, 401(k) balances, checking accounts, saving bonds, other interest-earning accounts, other interest-earning assets, stocks, and mutual funds less non mortgage debts. 

Among the $9915$ individuals, $3682$ are eligible to participate in the program. The variable *e401* indicates eligibility and *p401* indicates participation, respectively.

In [ ]:
data[!, "n"] .= 1;
e_count = combine(groupby(data, :e401), :n => sum => :Freq);
plot(e_count, color=:e401, y=:Freq, Geom.bar(position=:dodge))

Eligibility is highly associated with financial wealth:

In [ ]:
plot(data, x=:net_tfa, xgroup=:e401, color=:e401, Geom.subplot_grid(Geom.density()))

The unconditional APE of e401 is about $19559$

In [4]:
e1 = data[data.e401 .== 0, :];
e2 = data[data.e401 .== 1, :];
round(mean(e2.net_tfa) - mean(e1.net_tfa))

19559.0

Among the $3682$ individuals that are eligible, $2594$ decided to participate in the program. The unconditional APE of p401 is about $27372$

In [5]:
p1 = data[data.p401 .== 0, :];
p2 = data[data.p401 .== 1, :];
round(mean(p2.net_tfa) - mean(p1.net_tfa))

27372.0

As discussed, these estimates are biadsed since they don't account for saver heterogeneity and endogeneity of participation.

## Estimating the ATE of 401(k) Eligibility on Financial Assets

We first look at the treatment effect of e401 on net total financial assets. We give estimates of the ATE and ATT that corresponds to the linear model

$$
Y = D\alpha + f(X)'\beta + \epsilon,
$$

where  $f(X)$  includes indicators of marital status, two-earner status, defined benefit pension status, IRA participation status, and home ownership status, and orthogonal polynomials of degrees 2, 4, 6 and 8 in family size, education, age and income, respectively. The dimensions of  $f(X)$  is 25.

In the first step, we report estimates of the average treatment effect (ATE) of 401(k) eligibility on net financial assets both in the partially linear regression (PLR) model and in the interactive regression model (IRM) allowing for heterogeneous treatment effects.

In [6]:
# We define the `poly` function as provided by the documentation of the `StatsModels` package:

# syntax: best practice to define a _new_ function
poly(x, n) = x^n

# type of model where syntax applies: here this applies to any model type
const POLY_CONTEXT = Any

# struct for behavior
struct PolyTerm{T,D} <: AbstractTerm
    term::T
    deg::D
end

Base.show(io::IO, p::PolyTerm) = print(io, "poly($(p.term), $(p.deg))")

# for `poly` use at run-time (outside @formula), return a schema-less PolyTerm
poly(t::Symbol, d::Int) = PolyTerm(term(t), term(d))

# for `poly` use inside @formula: create a schemaless PolyTerm and apply_schema
function StatsModels.apply_schema(t::FunctionTerm{typeof(poly)},
                                  sch::StatsModels.Schema,
                                  Mod::Type{<:POLY_CONTEXT})
    apply_schema(PolyTerm(t.args_parsed...), sch, Mod)
end

# apply_schema to internal Terms and check for proper types
function StatsModels.apply_schema(t::PolyTerm,
                                  sch::StatsModels.Schema,
                                  Mod::Type{<:POLY_CONTEXT})
    term = apply_schema(t.term, sch, Mod)
    isa(term, ContinuousTerm) ||
        throw(ArgumentError("PolyTerm only works with continuous terms (got $term)"))
    isa(t.deg, ConstantTerm) ||
        throw(ArgumentError("PolyTerm degree must be a number (got $t.deg)"))
    PolyTerm(term, t.deg.n)
end

function StatsModels.modelcols(p::PolyTerm, d::NamedTuple)
    col = modelcols(p.term, d)
    reduce(hcat, [col.^n for n in 1:p.deg])
end

# the basic terms contained within a PolyTerm (for schema extraction)
StatsModels.terms(p::PolyTerm) = terms(p.term)
# names variables from the data that a PolyTerm relies on
StatsModels.termvars(p::PolyTerm) = StatsModels.termvars(p.term)
# number of columns in the matrix this term produces
StatsModels.width(p::PolyTerm) = p.deg

StatsBase.coefnames(p::PolyTerm) = coefnames(p.term) .* "^" .* string.(1:p.deg)

# output


In [52]:
#Constructing the Data

formula_flex = @formula(net_tfa ~ e401 + poly(age, 6) + poly(inc, 8) + poly(educ, 4) + poly(fsize, 2) + marr + twoearn + db + pira + hown);
formula_flex = apply_schema(formula_flex, schema(data));
y, x = modelcols(formula_flex, data);
y = Float64.(y)
d = x[:, 1];
x = x[:, Not(1)];
size(x, 2)

25

## Partially Linear Regression Models (PLR)

We start using lasso to estimate the function $g_0$ and $m_0$ in the following PLR model:

$$
Y = D\theta_0 + g_0(X) + \zeta, E[\zeta | D, X] = 0,
$$

$$
D = m_0(X) + V, E[V| X] = 0.
$$

In [60]:
using MLDataUtils, MLBase, Random, FixedEffectModels, GLMNet

function DML2_for_PLM(x , d , y, dreg , yreg , nfold)
    
    # Num ob observations
    nobser = size(x,1)
    
    # Define folds indices 
    foldid = collect(Kfold(size(x)[1], nfold))
    
    # Create array to save errors 
    ytil = ones(nobser)
    dtil = ones(nobser)
    
    dl = convert(Matrix{Float64}, [(d .< 0.5) (d .>= 0.5)])
    
    # loop to save results
    for i in 1:nfold
        
        # Lasso regression, excluding folds selected 
        dfit = dreg(x[foldid[i],:], dl[foldid[i], :])
        yfit = yreg(x[foldid[i],:], y[foldid[i]])
        
        # Predict estimates using the 
        dhat = GLMNet.predict(dfit, x[Not(foldid[i]),:], outtype = :prob)
        yhat = GLMNet.predict(yfit, x[Not(foldid[i]),:])
        
        # Save errors 
        dtil[Not(foldid[i])] = (d[Not(foldid[i])] - dhat)
        ytil[Not(foldid[i])] = (y[Not(foldid[i])] - yhat)
    end
    
    # Create dataframe 
    data = DataFrame(ytil = ytil, dtil = dtil)
    
    # OLS clustering at the County level
    rfit = fit(LinearModel, reshape(dtil, nobser, 1), ytil)
    # coef_est = coef(rfit)[2]
    # se = FixedEffectModels.coeftable(rfit).cols[2][2]

    # println(" coef (se) = ", coef_est ,"(",se,")")
    
    return rfit, data;
    
end

DML2_for_PLM (generic function with 1 method)

In [61]:
# Estimating the PLR

Random.seed!(123)
dreg(x,d) = glmnetcv(x, d, nfolds = 5, Binomial())
yreg(x,y) = glmnetcv(x, y, nfolds = 5)
lasso_fit, lasso_data = DML2_for_PLM(x, d, y, dreg, yreg, 3);
lasso_fit

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}:

Coefficients:
─────────────────────────────────────────────────────────────
      Coef.  Std. Error     t  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────
x1  6536.48     1250.23  5.23    <1e-06    4085.77    8987.19
─────────────────────────────────────────────────────────────


Let us check the predictive performance of the model.

In [62]:
# cross-fitted RMSE: outcome
lasso_y_rmse = sqrt(mean((lasso_data[!, 1] .- StatsBase.coef(lasso_fit)[1] * lasso_data[!, 2]) .^ 2))

55738.19944273792

In [63]:
# cross-fitted RMSE: treatment

lasso_d_rmse = sqrt(mean(lasso_data[!, 2] .^ 2));

println(lasso_d_rmse)

# cross-fitted ce: treatment

mean(ifelse.(d .- lasso_data[!, 2] .> 0.5, 1, 0) .!= d)

0.44775159135451154


0.31860816944024206

Then we repeat this proceedure for various machine learning methods.

In [20]:
# Random Forrest

using DecisionTree

function DML2_RF(z , d , y, dreg , yreg , nfold)
    
    # Num ob observations
    nobser = size(z,1)
    
    # Define folds indices
    foldid = collect(Kfold(size(z)[1], nfold))
    
    # Create array to save errors 
    ytil = ones(nobser)
    dtil = ones(nobser)
    
    # loop to save results
    for i in 1:nfold
        dfit = dreg(z[foldid[i],:], d[foldid[i]])
        yfit = yreg(z[foldid[i],:], y[foldid[i]])
        dhat = apply_forest(dfit,z[Not(foldid[i]),:])
        yhat = apply_forest(yfit,z[Not(foldid[i]),:])
        dtil[Not(foldid[i])]   = (d[Not(foldid[i])] - dhat)
        ytil[Not(foldid[i])]   = (y[Not(foldid[i])] - yhat)
    end
    
    # Create dataframe 
    data = DataFrame(ytil = ytil, dtil = dtil)
    
    # OLS clustering at the County level
    # rfit = reg(data, @formula(ytil ~ dtil))
    rfit = fit(LinearModel, reshape(dtil, nobser, 1), ytil)
    # coef_est = coef(rfit)[1]
    # se = FixedEffectModels.coeftable(rfit).cols[2]

    # println(" coef (se) = ", coef_est ,"(",se,")")
    
    return rfit, data;
    
end

DML2_RF (generic function with 1 method)

In [24]:
foldid = collect(Kfold(size(x)[1], 3))

3-element Vector{Any}:
 [2, 4, 6, 8, 9, 11, 12, 13, 14, 16  …  9901, 9903, 9904, 9905, 9908, 9909, 9910, 9911, 9914, 9915]
 [1, 3, 5, 7, 8, 9, 10, 12, 13, 15  …  9900, 9902, 9903, 9905, 9906, 9907, 9910, 9912, 9913, 9915]
 [1, 2, 3, 4, 5, 6, 7, 10, 11, 14  …  9902, 9904, 9906, 9907, 9908, 9909, 9911, 9912, 9913, 9914]

In [26]:
size(foldid[1], 1)

3305

In [21]:
Random.seed!(123)

dreg(x, d) = build_forest(d, x)
yreg(x, y) = build_forest(y, x)

rf_fit, rf_data = DML2_RF(x, d, y, dreg, yreg, 3);
rf_fit

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}:

Coefficients:
─────────────────────────────────────────────────────────────
      Coef.  Std. Error     t  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────
x1  8753.48      1214.5  7.21    <1e-12    6372.81    11134.2
─────────────────────────────────────────────────────────────


We can compare the accuracy of this model to the model that was estimated with lasso.

In [14]:
rf_y_rmse = sqrt(mean((rf_data[!, 1] .- StatsBase.coef(rf_fit)[1] * rf_data[!, 2]) .^ 2))
rf_d_rmse = sqrt(mean(rf_data[!, 2] .^ 2))

println(rf_y_rmse)

println(rf_d_rmse)

mean(ifelse.(d .- rf_data[!, 2] .> 0.5, 1, 0) .!= d)

56226.390040126214
0.46496145990094206

0.3447302067574382

In [15]:
# Trees

function DML2_Tree(z , d , y, dreg , yreg , nfold)
    
    # Num ob observations
    nobser = size(z,1)
    
    # Define folds indices
    foldid = collect(Kfold(size(z)[1], nfold))
    
    # Create array to save errors 
    ytil = ones(nobser)
    dtil = ones(nobser)
    
    # loop to save results
    for i in 1:nfold
        dfit = dreg(z[foldid[i],:], d[foldid[i]])
        yfit = yreg(z[foldid[i],:], y[foldid[i]])
        dhat = apply_tree(dfit,z[Not(foldid[i]),:])
        yhat = apply_tree(yfit,z[Not(foldid[i]),:])
        dtil[Not(foldid[i])]   = (d[Not(foldid[i])] - dhat)
        ytil[Not(foldid[i])]   = (y[Not(foldid[i])] - yhat)
    end
    
    # Create dataframe 
    data = DataFrame(ytil = ytil, dtil = dtil)
    
    # OLS clustering at the County level
    rfit = fit(LinearModel, reshape(dtil, nobser, 1), ytil)
    # coef_est = coef(rfit)[1]
    # se = FixedEffectModels.coeftable(rfit).cols[2]

    # println(" coef (se) = ", coef_est ,"(",se,")")
    
    return rfit, data;
    
end

DML2_Tree (generic function with 1 method)

In [16]:
Random.seed!(123)

dreg(x, d) = build_tree(d, x, 0, 30, 7, 20, 0.01)
yreg(x, y) = build_tree(y, x, 0, 30, 7, 20, 0.01)

tree_fit, tree_data = DML2_Tree(x, d, y, dreg, yreg, 3);
tree_fit

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}:

Coefficients:
─────────────────────────────────────────────────────────────
      Coef.  Std. Error     t  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────
x1  8205.33     1320.35  6.21    <1e-09    5617.18    10793.5
─────────────────────────────────────────────────────────────


In [17]:
tree_y_rmse = sqrt(mean((tree_data[!, 1] .- StatsBase.coef(tree_fit)[1] * tree_data[!, 2]) .^ 2))
tree_d_rmse = sqrt(mean(tree_data[!, 2] .^ 2))

println(tree_y_rmse)

println(tree_d_rmse)

mean(ifelse.(d .- tree_data[!, 2] .> 0.5, 1, 0) .!= d)

59676.120532917645
0.45392774217158094


0.34664649520927887

In [85]:
# Boosting

using XGBoost

function DML2_Boost(z , d , y, dreg , yreg , nfold)
    
    # Num ob observations
    nobser = size(z,1)
    
    # Define folds indices
    foldid = collect(Kfold(size(z)[1], nfold))
    
    # Create array to save errors 
    ytil = ones(nobser)
    dtil = ones(nobser)
    
    # loop to save results
    for i in 1:nfold
        dfit = dreg(z[foldid[i], :], d[foldid[i]])
        yfit = yreg(z[foldid[i], :], y[foldid[i]])
        dhat = XGBoost.predict(dfit, z[Not(foldid[i]), :])
        yhat = XGBoost.predict(yfit, z[Not(foldid[i]), :])
        dtil[Not(foldid[i])]   = (d[Not(foldid[i])] - dhat)
        ytil[Not(foldid[i])]   = (y[Not(foldid[i])] - yhat)
    end
    
    # Create dataframe 
    data = DataFrame(ytil = ytil, dtil = dtil)
    
    # OLS clustering at the County level
    rfit = fit(LinearModel, reshape(dtil, nobser, 1), ytil)
    # coef_est = coef(rfit)[1]
    # se = FixedEffectModels.coeftable(rfit).cols[2]

    # println(" coef (se) = ", coef_est ,"(",se,")")
    
    return rfit, data;
    
end

DML2_Boost (generic function with 1 method)

In [19]:
Random.seed!(123)

dreg(x, d) = xgboost(x, 5, label = d, objective = "binary:logistic", eval_metric = "logloss");
yreg(x, y) = xgboost(x, 5, label = y);

boost_fit, boost_data = DML2_Boost(x, d, y, dreg, yreg, 3);
boost_fit

[1]	train-logloss:0.62814512403397260
[2]	train-logloss:0.59143994323424598
[3]	train-logloss:0.56518430078399340
[4]	train-logloss:0.54780421759510545
[5]	train-logloss:0.53631460053477098
[1]	train-rmse:50805.90975126884586643
[2]	train-rmse:45118.49221384151314851
[3]	train-rmse:41081.95237535334308632
[4]	train-rmse:38870.17674265824462054
[5]	train-rmse:36808.78356694047397468
[1]	train-logloss:0.62360870221829812
[2]	train-logloss:0.58373274581785828
[3]	train-logloss:0.55822905044151683
[4]	train-logloss:0.54074468446977197
[5]	train-logloss:0.52591642424712204
[1]	train-rmse:60903.80710911499772919
[2]	train-rmse:54099.31892967657768168
[3]	train-rmse:49576.11850658095499966
[4]	train-rmse:46556.65778888474596897
[5]	train-rmse:43876.91871598918805830
[1]	train-logloss:0.62742250420592738
[2]	train-logloss:0.58906182237814853
[3]	train-logloss:0.56361801188760374
[4]	train-logloss:0.54553039387071367
[5]	train-logloss:0.53199355807634419
[1]	train-rmse:58283.80948005586833460
[

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}:

Coefficients:
─────────────────────────────────────────────────────────────
      Coef.  Std. Error     t  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────
x1  8469.37     1274.65  6.64    <1e-10    5970.79    10967.9
─────────────────────────────────────────────────────────────


In [20]:
boost_y_rmse = sqrt(mean((boost_data[!, 1] .- StatsBase.coef(boost_fit)[1] * boost_data[!, 2]) .^ 2))
boost_d_rmse = sqrt(mean(boost_data[!, 2] .^ 2))

println(boost_y_rmse)

println(boost_d_rmse)

mean(ifelse.(d .- boost_data[!, 2] .> 0.5, 1, 0) .!= d)

56917.35363267462
0.44846539610140396


0.3179021684316692

Let's sum up the results:

In [ ]:
DataFrame(Statistic = ["Estimate", "Std.Error", "RMSE Y", "RMSE D"], 
    Lasso = [StatsBase.coef(lasso_fit)[1], sqrt(vcov(lasso_fit)[1]), lasso_y_rmse, lasso_d_rmse], 
    RF = [StatsBase.coef(rf_fit)[1], sqrt(vcov(rf_fit)[1]), rf_y_rmse, rf_d_rmse], 
    Trees = [StatsBase.coef(tree_fit)[1], sqrt(vcov(tree_fit)[1]), tree_y_rmse, tree_d_rmse], 
    Boosting = [StatsBase.coef(boost_fit)[1], sqrt(vcov(boost_fit)[1]), boost_y_rmse, boost_d_rmse])

,Statistic,Lasso,RF,Trees,Boosting
,String,Float64,Float64,Float64,Float64
1,Estimate,6536.48,8753.48,8205.33,8469.37
2,Std.Error,1250.23,1214.5,1320.35,1274.65
3,RMSE Y,55738.2,56226.4,59676.1,56917.4
4,RMSE D,0.447752,0.464961,0.453928,0.448465


## Interactive Regression Model (IRM)

Next, we consider estimation of average treatment effects when treatment effects are fully heterogeneous:

$$
Y = g_0(D, X) + U, E[U | X, D] = 0,
$$

$$
D = m_0(X) + V, E[V | X] = 0.
$$

In [64]:
# Function based off of: https://github.com/DoubleML/doubleml-for-r/blob/f00d62c722a2b1e37c01b7f7f772e9d07f452a98/R/double_ml_irm.R
#                        https://github.com/DoubleML/doubleml-for-r/blob/f00d62c722a2b1e37c01b7f7f772e9d07f452a98/R/double_ml_plr.R

# Function takes x, y, d, learners, and nfolds

function IRM_Lasso(x, y, d, ml_g, ml_m, nfold, trimming_threshold = 1e-12)
    
    # Sample size
    nobser = size(x, 1)
    
    # Fold indexes
    foldid = collect(Kfold(size(x)[1], nfold))
    
    # Initialize vectors for predictions
    y1_hat = ones(nobser)
    y0_hat = ones(nobser)
    d_hat = ones(nobser)
    dl = convert(Matrix{Float64}, [(d .< 0.5) (d .>= 0.5)])
    
    # Apply learners to y_0, y_1 and d separately
    for i in 1:nfold
        # Create y_0 and y_1 for this fold
        mask = findall(==(1), d[foldid[i]])
        smp_1 = foldid[i][mask]
        smp_0 = foldid[i][Not(mask)]
        
        # Model Learning
        g0_hat = ml_g(x[smp_0, :], y[smp_0])
        g1_hat = ml_g(x[smp_1, :], y[smp_1])
        m_hat = ml_m(x[foldid[i], :], dl[foldid[i], :])
        
        # Predict: g0_hat, g1_hat, m_hat
        d_hat[Not(foldid[i])] = GLMNet.predict(m_hat, x[Not(foldid[i]), :], outtype = :prob)
        y0_hat[Not(foldid[i])] = GLMNet.predict(g0_hat, x[Not(foldid[i]), :])
        y1_hat[Not(foldid[i])] = GLMNet.predict(g1_hat, x[Not(foldid[i]), :])
    
    end
    
    # Residuals: u0_hat, u1_hat, no need for residual in d
    u0_hat = y .- y0_hat
    u1_hat = y .- y1_hat
    
    # Trimming
    d_hat[d_hat .< trimming_threshold] .= trimming_threshold
    d_hat[d_hat .> (1 - trimming_threshold)] .= 1 - trimming_threshold

    # Compute regression terms:
    # Left side: y1_hat - y0_hat + d * u1_hat / m_hat - (1 - d) * u0_hat / (1 - m_hat)
    psi_b = y1_hat .- y0_hat .+ d .* u1_hat ./ d_hat - (1 .- d) .* u0_hat ./ (1 .- d_hat)
    
    # Right side: All ones
    psi_a = reshape(ones(nobser), nobser, 1)
    
    # Regression with fit(LinearModel, ...)
    rfit = fit(LinearModel, psi_a, psi_b)
    
    # Generate data matrix for output
    u_hat = d .* u1_hat + (1 .- d) .* u0_hat
    d_til = d .- d_hat
    data = DataFrame(u_hat = u_hat, d_til = d_til)
    
    # Function outputs residual data and ATE
    return rfit, data;
    
end

IRM_Lasso (generic function with 2 methods)

In [65]:
Random.seed!(123)
ml_m(x, d) = glmnetcv(x, d, nfolds = 5, Binomial())
ml_g(x, y) = glmnetcv(x, y, nfolds = 5)
lasso_fit, lasso_data = IRM_Lasso(x, y, d, ml_g, ml_m, 3, 0.01);
lasso_fit

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}:

Coefficients:
────────────────────────────────────────────────────────────
     Coef.  Std. Error     t  Pr(>|t|)  Lower 95%  Upper 95%
────────────────────────────────────────────────────────────
x1  1527.2     4241.99  0.36    0.7188   -6787.97    9842.37
────────────────────────────────────────────────────────────


In [92]:
function IRM_Forest(x, y, d, ml_g, ml_m, nfold)
    
    # Sample size
    nobser = size(x, 1)
    
    # Fold indexes
    foldid = collect(Kfold(size(x)[1], nfold))
    
    # Initialize vectors for predictions
    y1_hat = ones(nobser)
    y0_hat = ones(nobser)
    d_hat = ones(nobser)
    
    # Apply learners to y_0, y_1 and d separately
    for i in 1:nfold
        # Create y_0 and y_1 for this fold
        mask = findall(==(1), d[foldid[i]])
        smp_1 = foldid[i][mask]
        smp_0 = foldid[i][Not(mask)]
        
        # Model Learning
        g0_hat = ml_g(x[smp_0, :], y[smp_0])
        g1_hat = ml_g(x[smp_1, :], y[smp_1])
        m_hat = ml_m(x[foldid[i], :], d[foldid[i]])
        
        # Predict: g0_hat, g1_hat, m_hat
        d_hat[Not(foldid[i])] = apply_forest(m_hat, x[Not(foldid[i]), :])
        y0_hat[Not(foldid[i])] = apply_forest(g0_hat, x[Not(foldid[i]), :])
        y1_hat[Not(foldid[i])] = apply_forest(g1_hat, x[Not(foldid[i]), :])
    
    end
    
    # Residuals: u0_hat, u1_hat, no need for residual in d
    u0_hat = y .- y0_hat
    u1_hat = y .- y1_hat
    
    # Compute regression terms:
    # Left side: y1_hat - y0_hat + d * u1_hat / m_hat 
    #            - (1 - d) * u0_hat / (1 - m_hat)
    psi_b = y1_hat .- y0_hat .+ d .* u1_hat ./ d_hat - (1 .- d) .* u0_hat ./ (1 .- d_hat)
    
    # Right side: All ones
    psi_a = reshape(ones(nobser), nobser, 1)
    
    # Regression with fit(LinearModel, ...)
    rfit = fit(LinearModel, psi_a, psi_b)
    
    # Generate data matrix for output
    u_hat = d .* u1_hat + (1 .- d) .* u0_hat
    d_til = d .- d_hat
    data = DataFrame(u_hat = u_hat, d_til = d_til)
    
    # Function outputs residual data and ATE
    return rfit, data, psi_a, psi_b;
    
end

IRM_Forest (generic function with 1 method)

In [116]:
Random.seed!(123)

ml_m(x, d) = build_forest(d, x)
ml_g(x, y) = build_forest(y, x)

rf_fit, rf_data, psi_a, psi_b = IRM_Forest(x, y, d, ml_g, ml_m, 3);
rf_fit

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}:

Coefficients:
──────────────────────────────────────────────────────────
    Coef.  Std. Error    t  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────
x1    NaN         NaN  NaN       NaN        NaN        NaN
──────────────────────────────────────────────────────────


In [170]:
psi_b

9915-element Vector{Float64}:
 NaN
 NaN
 NaN
 NaN
 NaN
  Inf
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
   ⋮
  Inf
  Inf
 -Inf
  Inf
  Inf
 -Inf
  Inf
 NaN
 NaN
 NaN
 -Inf
 -Inf

In [95]:
function IRM_Tree(x, y, d, ml_g, ml_m, nfold)
    
    # Sample size
    nobser = size(x, 1)
    
    # Fold indexes
    foldid = collect(Kfold(size(x)[1], nfold))
    
    # Initialize vectors for predictions
    y1_hat = ones(nobser)
    y0_hat = ones(nobser)
    d_hat = ones(nobser)
    
    # Apply learners to y_0, y_1 and d separately
    for i in 1:nfold
        # Create y_0 and y_1 for this fold
        mask = findall(==(1), d[foldid[i]])
        smp_1 = foldid[i][mask]
        smp_0 = foldid[i][Not(mask)]
        
        # Model Learning
        g0_hat = ml_g(x[smp_0, :], y[smp_0])
        g1_hat = ml_g(x[smp_1, :], y[smp_1])
        m_hat = ml_m(x[foldid[i], :], d[foldid[i]])
        
        # Predict: g0_hat, g1_hat, m_hat
        d_hat[Not(foldid[i])] = apply_tree(m_hat, x[Not(foldid[i]), :])
        y0_hat[Not(foldid[i])] = apply_tree(g0_hat, x[Not(foldid[i]), :])
        y1_hat[Not(foldid[i])] = apply_tree(g1_hat, x[Not(foldid[i]), :])
    
    end
    
    # Residuals: u0_hat, u1_hat, no need for residual in d
    u0_hat = y .- y0_hat
    u1_hat = y .- y1_hat
    
    # Compute regression terms:
    # Left side: y1_hat - y0_hat + d * u1_hat / m_hat 
    #            - (1 - d) * u0_hat / (1 - m_hat)
    psi_b = y1_hat .- y0_hat .+ d .* u1_hat ./ d_hat - (1 .- d) .* u0_hat ./ (1 .- d_hat)
    
    # Right side: All ones
    psi_a = reshape(ones(nobser), nobser, 1)
    
    # Regression with fit(LinearModel, ...)
    rfit = fit(LinearModel, psi_a, psi_b)
    
    # Generate data matrix for output
    u_hat = d .* u1_hat + (1 .- d) .* u0_hat
    d_til = d .- d_hat
    data = DataFrame(u_hat = u_hat, d_til = d_til)
    
    # Function outputs residual data and ATE
    return rfit, data;
    
end

IRM_Tree (generic function with 2 methods)

In [97]:
Random.seed!(123)

ml_m(x, d) = build_tree(d, x, 0, 30, 7, 20, 0.01)
ml_g(x, y) = build_tree(y, x, 0, 30, 7, 20, 0.01)

tree_fit, tree_data = IRM_Tree(x, y, d, ml_g, ml_m, 3);
tree_fit

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}:

Coefficients:
─────────────────────────────────────────────────────────────
      Coef.  Std. Error     t  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────
x1  7137.53     1212.17  5.89    <1e-08    4761.43    9513.63
─────────────────────────────────────────────────────────────


In [86]:

function IRM_Boost(x, y, d, ml_g, ml_m, nfold, trimming_threshold = 1e-12)
    
    # Sample size
    nobser = size(x, 1)
    
    # Fold indexes
    foldid = collect(Kfold(size(x)[1], nfold))
    
    # Initialize vectors for predictions
    y1_hat = ones(nobser)
    y0_hat = ones(nobser)
    d_hat = ones(nobser)
    
    # Apply learners to y_0, y_1 and d separately
    for i in 1:nfold
        # Create y_0 and y_1 for this fold
        mask = findall(==(1), d[foldid[i]])
        smp_1 = foldid[i][mask]
        smp_0 = foldid[i][Not(mask)]
        
        # Model Learning
        g0_hat = ml_g(x[smp_0, :], y[smp_0])
        g1_hat = ml_g(x[smp_1, :], y[smp_1])
        m_hat = ml_m(x[foldid[i], :], d[foldid[i]])
        
        # Predict: g0_hat, g1_hat, m_hat
        d_hat[Not(foldid[i])] = XGBoost.predict(m_hat, x[Not(foldid[i]), :])
        y0_hat[Not(foldid[i])] = XGBoost.predict(g0_hat, x[Not(foldid[i]), :])
        y1_hat[Not(foldid[i])] = XGBoost.predict(g1_hat, x[Not(foldid[i]), :])
    
    end
    
    # Residuals: u0_hat, u1_hat, no need for residual in d
    u0_hat = y .- y0_hat
    u1_hat = y .- y1_hat
    
    # Trimming
    d_hat[d_hat .< trimming_threshold] .= trimming_threshold
    d_hat[d_hat .> (1 - trimming_threshold)] .= 1 - trimming_threshold

    # Compute regression terms:
    # Left side: y1_hat - y0_hat + d * u1_hat / m_hat 
    #            - (1 - d) * u0_hat / (1 - m_hat)
    psi_b = y1_hat .- y0_hat .+ d .* u1_hat ./ d_hat - (1 .- d) .* u0_hat ./ (1 .- d_hat)
    
    # Right side: All ones
    psi_a = reshape(ones(nobser), nobser, 1)
    
    # Regression with fit(LinearModel, ...)
    rfit = fit(LinearModel, psi_a, psi_b)
    
    # Generate data matrix for output
    u_hat = d .* u1_hat + (1 .- d) .* u0_hat
    d_til = d .- d_hat
    data = DataFrame(u_hat = u_hat, d_til = d_til)
    
    # Function outputs residual data and ATE
    return rfit, data, psi_a, psi_b;
    
end

IRM_Boost (generic function with 2 methods)

In [109]:
Random.seed!(123)

ml_m(x, d) = xgboost(x, 5, label = d, objective = "binary:logistic", eval_metric = "logloss");
ml_g(x, y) = xgboost(x, 5, label = y);

boost_fit, boost_data, psi_a, psi_b = IRM_Boost(x, y, d, ml_g, ml_m, 3, 0.01);
boost_fit

[1]	train-rmse:39672.55862477455229964
[2]	train-rmse:33829.13558642176212743
[3]	train-rmse:30258.46958888950393884
[4]	train-rmse:27771.90457050152326701
[5]	train-rmse:25801.52380595109571004
[1]	train-rmse:63219.98147018595773261
[2]	train-rmse:54416.80836068426287966
[3]	train-rmse:47289.68965621231473051
[4]	train-rmse:42172.63499183114618063
[5]	train-rmse:37971.38256453725625761
[1]	train-logloss:0.62814512403397260
[2]	train-logloss:0.59143994323424598
[3]	train-logloss:0.56518430078399340
[4]	train-logloss:0.54780421759510545
[5]	train-logloss:0.53631460053477098
[1]	train-rmse:50636.30439475246384973
[2]	train-rmse:43745.13860136931907618
[3]	train-rmse:38834.36631127876171377
[4]	train-rmse:35144.75589004970242968
[5]	train-rmse:32663.59513701907417271
[1]	train-rmse:73605.79290901409694925
[2]	train-rmse:65207.46431023348122835
[3]	train-rmse:58405.75253156274266075
[4]	train-rmse:53038.06686622917186469
[5]	train-rmse:49754.44319479364639847
[1]	train-logloss:0.6236087022

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}:

Coefficients:
─────────────────────────────────────────────────────────────
      Coef.  Std. Error     t  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────
x1  7955.78      1175.3  6.77    <1e-10    5651.95    10259.6
─────────────────────────────────────────────────────────────


## Local Average Treatment Effects of 401(k) Participation on Net Financial Assets

## Interactive IV Model (IIVM)

Now, we consider estimation of local average treatment effects (LATE) with the binary instrument 401. As before, $Y$ denotes the outcome `net_tfa`, and $X$ is the vector of covariates. Here, the structural equation model is:

$$
Y = g_0(Z, X) + U, E[U | Z, X] = 0,
$$

$$
D = r_0(Z, X) + V, E[V | Z, X] = 0,
$$

$$
Z = m_0(X) + \zeta, E[\zeta | X] = 0.
$$

In [121]:
formula_iivm = @formula(net_tfa ~ p401 + e401 + poly(age, 6) + poly(inc, 8) + poly(educ, 4) + poly(fsize, 2) + marr + twoearn + db + pira + hown);
formula_iivm = apply_schema(formula_iivm, schema(data));
y, x = modelcols(formula_iivm, data);
d = Integer.(x[:, 2]);
z = Integer.(x[:, 1]);
x = x[:, Not([1, 2])];
size(x, 2)

25

In [145]:
function IIVM_Lasso(x, y, d, z, ml_g, ml_r, ml_m, nfold, trimming_threshold = 1e-12)
    
    # Sample size
    nobser = size(x, 1)
    
    # Fold indexes
    foldid = collect(Kfold(size(x)[1], nfold))
    
    # Initialize vectors for predictions
    y0_hat = ones(nobser)
    y1_hat = ones(nobser)
    d0_hat = zeros(nobser)
    d1_hat = ones(nobser)
    z_hat = ones(nobser)
    
    dl = convert(Matrix{Float64}, [(d .< 0.5) (d .>= 0.5)])
    zl = convert(Matrix{Float64}, [(z .< 0.5) (z .>= 0.5)])
    
    # Apply learners to y_0, y_1, d_1, d_2 and d separately
    for i in 1:nfold
        # Create y_0 and y_1 for this fold
        mask = findall(==(1), z[foldid[i]])
        smp_1 = foldid[i][mask]
        smp_0 = foldid[i][Not(mask)]
        
        # Model Learning
        g0_hat = ml_g(x[smp_0, :], y[smp_0])
        g1_hat = ml_g(x[smp_1, :], y[smp_1])
        r1_hat = ml_r(x[smp_1, :], dl[smp_1, :])
        m_hat = ml_m(x[foldid[i], :], zl[foldid[i], :])
        
        # Predict: g0_hat, g1_hat, m_hat
        d1_hat[Not(foldid[i])] = GLMNet.predict(r1_hat, x[Not(foldid[i]), :], outtype = :prob)
        y0_hat[Not(foldid[i])] = GLMNet.predict(g0_hat, x[Not(foldid[i]), :])
        y1_hat[Not(foldid[i])] = GLMNet.predict(g1_hat, x[Not(foldid[i]), :])
        z_hat[Not(foldid[i])] = GLMNet.predict(m_hat, x[Not(foldid[i]), :], outtype = :prob)
    
    end
    
    
    # Residuals: u0_hat, u1_hat, w0_hat, and w1_hat; no need for residual in z
    u0_hat = y .- y0_hat
    u1_hat = y .- y1_hat
    w0_hat = d .- d0_hat
    w1_hat = d .- d1_hat
    
    # Compute regression terms:
    # Left side: y1_hat - y0_hat + z * (u1_hat) / z_hat - (1 - z) * u0_hat / (1 - z_hat)
    psi_b = y1_hat .- y0_hat .+ z .* u1_hat ./ z_hat .- (1 .- z) .* u0_hat ./ (1 .- z_hat)
    
    # Right side: d1_hat - d0_hat + z * (w1_hat) / z_hat - (1 - z) * w0_hat / (1 - z_hat)
    psi_a = d1_hat .- d0_hat .+ z .* w1_hat ./ z_hat .- (1 .- z) .* w0_hat ./ (1 .- z_hat)
    
    # Regression with fit(LinearModel, ...)
    rfit = fit(LinearModel, reshape(psi_a, nobser, 1), psi_b)
    
    # Generate data matrix for output
    u_hat = z .* u1_hat + (1 .- z) .* u0_hat
    d_til = z .* w1_hat + (1 .- z) .* w0_hat
    data = DataFrame(u_hat = u_hat, d_til = d_til)
    
    # Function outputs residual data and ATE
    return rfit, data, psi_a, psi_b;
    
end

IIVM_Lasso (generic function with 2 methods)

In [146]:
Random.seed!(123)

ml_r(x, d) = glmnetcv(x, d, nfolds = 5, Binomial());
ml_g(x, y) = glmnetcv(x, y, nfolds = 5);
ml_m(x, z) = glmnetcv(x, z, nfolds = 5, Binomial());

boost_fit, boost_data, psi_a, psi_b = IIVM_Boost(x, y, d, z, ml_g, ml_r, ml_m, 3)
boost_fit

LoadError: MethodError: no method matching glmnet!(::Matrix{Float64}, ::Vector{Float64}, ::Binomial{Float64}; weights=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], offsets=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
[0mClosest candidates are:
[0m  glmnet!(::Matrix{Float64}, [91m::Matrix{Float64}[39m, ::Binomial; offsets, weights, alpha, penalty_factor, constraints, dfmax, pmax, nlambda, lambda_min_ratio, lambda, tol, standardize, intercept, maxit, algorithm) at C:\Users\Work\.julia\packages\GLMNet\C8WKF\src\GLMNet.jl:337
[0m  glmnet!(::Matrix{Float64}, ::Vector{Float64}) at C:\Users\Work\.julia\packages\GLMNet\C8WKF\src\GLMNet.jl:271[91m got unsupported keyword arguments "weights", "offsets"[39m
[0m  glmnet!(::Matrix{Float64}, ::Vector{Float64}, [91m::Normal[39m; weights, naivealgorithm, alpha, penalty_factor, constraints, dfmax, pmax, nlambda, lambda_min_ra

In [150]:
function IIVM_Forest(x, y, d, z, ml_g, ml_r, ml_m, nfold, trimming_threshold = 1e-12)
    
    # Sample size
    nobser = size(x, 1)
    
    # Fold indexes
    foldid = collect(Kfold(size(x)[1], nfold))
    
    # Initialize vectors for predictions
    y0_hat = ones(nobser)
    y1_hat = ones(nobser)
    d0_hat = zeros(nobser)
    d1_hat = ones(nobser)
    z_hat = ones(nobser)
    
    # Apply learners to y_0, y_1, d_1, d_2 and d separately
    for i in 1:nfold
        # Create y_0 and y_1 for this fold
        mask = findall(==(1), z[foldid[i]])
        smp_1 = foldid[i][mask]
        smp_0 = foldid[i][Not(mask)]
        
        # Model Learning
        g0_hat = ml_g(x[smp_0, :], y[smp_0])
        g1_hat = ml_g(x[smp_1, :], y[smp_1])
        r1_hat = ml_r(x[smp_1, :], d[smp_1])
        m_hat = ml_m(x[foldid[i], :], z[foldid[i]])
        
        # Predict: g0_hat, g1_hat, m_hat
        d1_hat[Not(foldid[i])] = apply_forest(r1_hat, x[Not(foldid[i]), :])
        y0_hat[Not(foldid[i])] = apply_forest(g0_hat, x[Not(foldid[i]), :])
        y1_hat[Not(foldid[i])] = apply_forest(g1_hat, x[Not(foldid[i]), :])
        z_hat[Not(foldid[i])] = apply_forest(m_hat, x[Not(foldid[i]), :])
    
    end
    
    
    # Residuals: u0_hat, u1_hat, w0_hat, and w1_hat; no need for residual in z
    u0_hat = y .- y0_hat
    u1_hat = y .- y1_hat
    w0_hat = d .- d0_hat
    w1_hat = d .- d1_hat
    
    # Compute regression terms:
    # Left side: y1_hat - y0_hat + z * (u1_hat) / z_hat - (1 - z) * u0_hat / (1 - z_hat)
    psi_b = y1_hat .- y0_hat .+ z .* u1_hat ./ z_hat .- (1 .- z) .* u0_hat ./ (1 .- z_hat)
    
    # Right side: d1_hat - d0_hat + z * (w1_hat) / z_hat - (1 - z) * w0_hat / (1 - z_hat)
    psi_a = d1_hat .- d0_hat .+ z .* w1_hat ./ z_hat .- (1 .- z) .* w0_hat ./ (1 .- z_hat)
    
    # Regression with fit(LinearModel, ...)
    rfit = fit(LinearModel, reshape(psi_a, nobser, 1), psi_b)
    
    # Generate data matrix for output
    u_hat = z .* u1_hat + (1 .- z) .* u0_hat
    d_til = z .* w1_hat + (1 .- z) .* w0_hat
    data = DataFrame(u_hat = u_hat, d_til = d_til)
    
    # Function outputs residual data and ATE
    return rfit, data, psi_a, psi_b;
    
end

IIVM_Forest (generic function with 2 methods)

In [151]:
Random.seed!(123)

ml_r(x, d) = build_forest(d, x);
ml_g(x, y) = build_forest(y, x);
ml_m(x, z) = build_forest(z, x);

boost_fit, boost_data, psi_a, psi_b = IIVM_Forest(x, y, d, z, ml_g, ml_r, ml_m, 3)
boost_fit

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}:

Coefficients:
──────────────────────────────────────────────────────────
    Coef.  Std. Error    t  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────
x1    0.0         NaN  NaN       NaN        NaN        NaN
──────────────────────────────────────────────────────────


In [154]:
function IIVM_Tree(x, y, d, z, ml_g, ml_r, ml_m, nfold, trimming_threshold = 1e-12)
    
    # Sample size
    nobser = size(x, 1)
    
    # Fold indexes
    foldid = collect(Kfold(size(x)[1], nfold))
    
    # Initialize vectors for predictions
    y0_hat = ones(nobser)
    y1_hat = ones(nobser)
    d0_hat = zeros(nobser)
    d1_hat = ones(nobser)
    z_hat = ones(nobser)
    
    # Apply learners to y_0, y_1, d_1, d_2 and d separately
    for i in 1:nfold
        # Create y_0 and y_1 for this fold
        mask = findall(==(1), z[foldid[i]])
        smp_1 = foldid[i][mask]
        smp_0 = foldid[i][Not(mask)]
        
        # Model Learning
        g0_hat = ml_g(x[smp_0, :], y[smp_0])
        g1_hat = ml_g(x[smp_1, :], y[smp_1])
        r1_hat = ml_r(x[smp_1, :], d[smp_1])
        m_hat = ml_m(x[foldid[i], :], z[foldid[i]])
        
        # Predict: g0_hat, g1_hat, m_hat
        d1_hat[Not(foldid[i])] = apply_tree(r1_hat, x[Not(foldid[i]), :])
        y0_hat[Not(foldid[i])] = apply_tree(g0_hat, x[Not(foldid[i]), :])
        y1_hat[Not(foldid[i])] = apply_tree(g1_hat, x[Not(foldid[i]), :])
        z_hat[Not(foldid[i])] = apply_tree(m_hat, x[Not(foldid[i]), :])
    
    end
    
    
    # Residuals: u0_hat, u1_hat, w0_hat, and w1_hat; no need for residual in z
    u0_hat = y .- y0_hat
    u1_hat = y .- y1_hat
    w0_hat = d .- d0_hat
    w1_hat = d .- d1_hat
    
    # Compute regression terms:
    # Left side: y1_hat - y0_hat + z * (u1_hat) / z_hat - (1 - z) * u0_hat / (1 - z_hat)
    psi_b = y1_hat .- y0_hat .+ z .* u1_hat ./ z_hat .- (1 .- z) .* u0_hat ./ (1 .- z_hat)
    
    # Right side: d1_hat - d0_hat + z * (w1_hat) / z_hat - (1 - z) * w0_hat / (1 - z_hat)
    psi_a = d1_hat .- d0_hat .+ z .* w1_hat ./ z_hat .- (1 .- z) .* w0_hat ./ (1 .- z_hat)
    
    # Regression with fit(LinearModel, ...)
    rfit = fit(LinearModel, reshape(psi_a, nobser, 1), psi_b)
    
    # Generate data matrix for output
    u_hat = z .* u1_hat + (1 .- z) .* u0_hat
    d_til = z .* w1_hat + (1 .- z) .* w0_hat
    data = DataFrame(u_hat = u_hat, d_til = d_til)
    
    # Function outputs residual data and ATE
    return rfit, data, psi_a, psi_b;
    
end

IIVM_Tree (generic function with 2 methods)

In [155]:
Random.seed!(123)

ml_r(x, d) = build_tree(d, x, 0, 30, 7, 20, 0.01);
ml_g(x, y) = build_tree(y, x, 0, 30, 7, 20, 0.01);
ml_m(x, z) = build_tree(z, x, 0, 30, 7, 20, 0.01);

boost_fit, boost_data, psi_a, psi_b = IIVM_Tree(x, y, d, z, ml_g, ml_r, ml_m, 3)
boost_fit

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}:

Coefficients:
──────────────────────────────────────────────────────────
    Coef.  Std. Error    t  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────
x1    0.0         NaN  NaN       NaN        NaN        NaN
──────────────────────────────────────────────────────────


In [169]:
psi_b

9915-element Vector{Float64}:
 NaN
 NaN
 NaN
 NaN
 NaN
  Inf
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
   ⋮
  Inf
  Inf
 -Inf
  Inf
  Inf
 -Inf
  Inf
 NaN
 NaN
 NaN
 -Inf
 -Inf

In [139]:
function IIVM_Boost(x, y, d, z, ml_g, ml_r, ml_m, nfold, trimming_threshold = 1e-12)
    
    # Sample size
    nobser = size(x, 1)
    
    # Fold indexes
    foldid = collect(Kfold(size(x)[1], nfold))
    
    # Initialize vectors for predictions
    y0_hat = ones(nobser)
    y1_hat = ones(nobser)
    d0_hat = zeros(nobser)
    d1_hat = ones(nobser)
    z_hat = ones(nobser)
    
    # Apply learners to y_0, y_1, d_1, d_2 and d separately
    for i in 1:nfold
        # Create y_0 and y_1 for this fold
        mask = findall(==(1), z[foldid[i]])
        smp_1 = foldid[i][mask]
        smp_0 = foldid[i][Not(mask)]
        
        # Model Learning
        g0_hat = ml_g(x[smp_0, :], y[smp_0])
        g1_hat = ml_g(x[smp_1, :], y[smp_1])
        r1_hat = ml_r(x[smp_1, :], d[smp_1])
        m_hat = ml_m(x[foldid[i], :], z[foldid[i]])
        
        # Predict: g0_hat, g1_hat, m_hat
        d1_hat[Not(foldid[i])] = XGBoost.predict(r1_hat, x[Not(foldid[i]), :])
        y0_hat[Not(foldid[i])] = XGBoost.predict(g0_hat, x[Not(foldid[i]), :])
        y1_hat[Not(foldid[i])] = XGBoost.predict(g1_hat, x[Not(foldid[i]), :])
        z_hat[Not(foldid[i])] = XGBoost.predict(m_hat, x[Not(foldid[i]), :])
    
    end
    
    
    # Residuals: u0_hat, u1_hat, w0_hat, and w1_hat; no need for residual in z
    u0_hat = y .- y0_hat
    u1_hat = y .- y1_hat
    w0_hat = d .- d0_hat
    w1_hat = d .- d1_hat
    
    # Compute regression terms:
    # Left side: y1_hat - y0_hat + z * (u1_hat) / z_hat - (1 - z) * u0_hat / (1 - z_hat)
    psi_b = y1_hat .- y0_hat .+ z .* u1_hat ./ z_hat .- (1 .- z) .* u0_hat ./ (1 .- z_hat)
    
    # Right side: d1_hat - d0_hat + z * (w1_hat) / z_hat - (1 - z) * w0_hat / (1 - z_hat)
    psi_a = d1_hat .- d0_hat .+ z .* w1_hat ./ z_hat .- (1 .- z) .* w0_hat ./ (1 .- z_hat)
    
    # Regression with fit(LinearModel, ...)
    rfit = fit(LinearModel, reshape(psi_a, nobser, 1), psi_b)
    
    # Generate data matrix for output
    u_hat = z .* u1_hat + (1 .- z) .* u0_hat
    d_til = z .* w1_hat + (1 .- z) .* w0_hat
    data = DataFrame(u_hat = u_hat, d_til = d_til)
    
    # Function outputs residual data and ATE
    return rfit, data, psi_a, psi_b;
    
end

IIVM_Boost (generic function with 2 methods)

In [140]:
Random.seed!(123)

ml_r(x, d) = xgboost(x, 5, label = d, objective = "binary:logistic", eval_metric = "logloss");
ml_g(x, y) = xgboost(x, 5, label = y);
ml_m(x, z) = xgboost(x, 5, label = z, objective = "binary:logistic", eval_metric = "logloss");

boost_fit, boost_data, psi_a, psi_b = IIVM_Boost(x, y, d, z, ml_g, ml_r, ml_m, 3)
boost_fit

[1]	train-rmse:38737.01316460889211157
[2]	train-rmse:33304.54192929357668618
[3]	train-rmse:30087.88246757425804390
[4]	train-rmse:27682.53218891132928547
[5]	train-rmse:25969.88210150778104435
[1]	train-rmse:72572.35164468456059694
[2]	train-rmse:60784.81608743120159488
[3]	train-rmse:52507.22102924608770991
[4]	train-rmse:45813.72014589509490179
[5]	train-rmse:41249.85934676897886675
[1]	train-logloss:0.43798348307609558
[2]	train-logloss:0.29688626527786255
[3]	train-logloss:0.20793505012989044
[4]	train-logloss:0.14840467274188995
[5]	train-logloss:0.10719931125640869
[1]	train-logloss:0.60419132046728374
[2]	train-logloss:0.55244129131042286
[3]	train-logloss:0.52205823458584644
[4]	train-logloss:0.50040258928209136
[5]	train-logloss:0.48396389952180247
[1]	train-rmse:52097.06435315485578030
[2]	train-rmse:45438.81816587039793376
[3]	train-rmse:40529.84053594780561980
[4]	train-rmse:36828.59269390138797462
[5]	train-rmse:34190.20507247609930346
[1]	train-rmse:76699.46711005464021

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}:

Coefficients:
─────────────────────────────────────────────────────────────
      Coef.  Std. Error     t  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────
x1  12176.7     1268.88  9.60    <1e-20    9689.49    14664.0
─────────────────────────────────────────────────────────────


In [174]:
model = DecisionTreeClassifier(max_depth=2)
DecisionTree.fit!(model, x, d)

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  [0, 1]
root:                     Decision Tree
Leaves: 4
Depth:  2

In [181]:
predict_proba(model, x)[:, 2]

9915-element Vector{Float64}:
 0.15991116046640755
 0.43381136238279094
 0.43381136238279094
 0.43381136238279094
 0.6444823663253697
 0.6444823663253697
 0.6444823663253697
 0.15991116046640755
 0.15991116046640755
 0.15991116046640755
 0.6444823663253697
 0.43381136238279094
 0.43381136238279094
 ⋮
 0.43381136238279094
 0.43381136238279094
 0.43381136238279094
 0.15991116046640755
 0.15991116046640755
 0.43381136238279094
 0.43381136238279094
 0.43057050592034446
 0.6444823663253697
 0.43381136238279094
 0.6444823663253697
 0.15991116046640755

In [180]:
DecisionTree.predict(model, x)

9915-element Vector{Int64}:
 0
 0
 0
 0
 1
 1
 1
 0
 0
 0
 1
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 1
 0
 1
 0

In [182]:
print_tree(model)

Feature 7 < 29000.0 ?
├─ Feature 23 < 0.5 ?
    ├─ 0 : 3026/3602
    └─ 0 : 529/929
└─ Feature 23 < 0.5 ?
    ├─ 0 : 2053/3626
    └─ 1 : 1133/1758
